In [2]:
import json
from collections import Counter, defaultdict
from wiki_labels import qid_lab_get, qid_en_desc_get
from wiki_location import q2cc
from text_utils import cl
import sys
import gzip
from tqdm.notebook import tqdm

print(qid_lab_get(42, 'th').keys())


dict_keys(['ดักลัส แอดัมส์'])


In [4]:
from pythainlp import thai_letters
thai_letters = set(thai_letters)
t = pd.read_csv('/projekti/thainame/data/notable/wiki.csv.gz')
t

,Unnamed: 0,qid,name_nl,name_en,fn,ln,gender,cc_first,native_lang,ccs,desc_en,desc_nl,dob,dod,birth_place,birth_cc,death_place,death_cc,image,sort
0,829,45209,พูนสวัสดิ์ กระทิงแดงยิม,poonsawat kratingdaenggym,พูนสวัสดิ์,กระทิงแดงยิม,m,TH,th,"[('TH', 3)]",professional boxer from Thailand,นักมวยสากล,1980.0,NaN,phu phan,TH,NaN,NaN,Poonsawat Kratingdanggym.jpg,406687.0
1,992,52006,เป็นเอก การะเกตุ,pen-ek karaket,เป็นเอก,การะเกตุ,m,TH,th,"[('TH', 3)]",Thai taekwondo practitioner,NaN,1990.0,NaN,bangkok,TH,NaN,NaN,NaN,509106.0
2,1174,57670,ยิ่งลักษณ์ ชินวัตร,yingluck shinawatra,NaN,ชินวัตร,f,TH,th,"[('TH', 6), ('RS', 2), ('US', 1)]",Prime Minister of Thailand from 2011 to 2014,นายกรัฐมนตรีไทย,1967.0,NaN,san kamphaeng,TH,NaN,NaN,"Yingluck Shinawatra at US Embassy, Bangkok, Ju...",26361.0
3,2348,74861,ธานินทร์ กรัยวิเชียร,thanin kraivichien,ธานินทร์,กรัยวิเชียร,m,TH,th,"[('TH', 5), ('GB', 1)]",Thai lawyer and politician,ผู้พิพากษา,1927.0,NaN,bangkok,TH,NaN,NaN,Tanin2011.jpg,70201.0
4,2706,78455,พระเจนดุริยางค์,phra chenduriyang,NaN,NaN,m,TH,th,"[('TH', 5)]",Thai nobleman,คีตกวี,1883.0,NaN,phra nakhon,TH,bangkok,TH,พระเจนดุริยางค์ (ปิติ วาทยะกร).jpg,283068.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12493,10725067,118963611,อรถัสญาน์ สุกใส,alraphatsaya suksai,อรถัสญาน์,สุกใส,f,TH,th,"[('TH', 3)]",Thai female singer,นักร้อง,1995.0,NaN,mueang chanthaburi,TH,NaN,NaN,NaN,100001985.0
12494,10725114,118978364,kamonchanok seeken,kamonchanok seeken,kamonchanok,seeken,f,TH,th,"[('TH', 2)]",Thai taekwondo practitioner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100001970.0
12495,10732528,120028407,sirinart sugandharat,sirinart sugandharat,sirinart,sugandharat,f,TH,th,"[('TH', 4)]",Thai actress and model,นักแสดง,1995.0,NaN,saraburi,TH,NaN,NaN,NaN,100001985.0
12496,10733448,120246677,jo lindner,jo lindner,jo,lindner,m,TH,th,"[('TH', 1)]",b,NaN,1993.0,NaN,NaN,NaN,bangkok,TH,NaN,100001993.0


In [65]:
labels = defaultdict(Counter)

for i, l in tqdm(enumerate(gzip.open('/backup/wikidata/wikinelma.jsonl.gz')), total=25_042_218):
    j = json.loads(l)
    qid = int(j['wiki_id'][1:])
    if qid == 2108126:
        break
    elif False:
        th = False
        if 'native_language' in j and 'WIKI_Q9217' in j['native_language']:
            th = True
        elif 'name_native' in j and j['name_native'] and j['name_native'][0][0] in thai_letters:
            th = True
            
        elif 'country' in j and 'WIKI_Q869' in j['country']:
            th = True
        elif 'nationality' in j and 'WIKI_Q869' in j['nationality']:
            th = True
        labs = list(qid_lab_get(qid, 'th', include_alt=True).keys())
        if 'name_native' in j and j['name_native'] and j['name_native'][0][0] in thai_letters:
            labs += j['name_native']
        
        typ = j['type'] + ('_th' if th else '')
        for l in labs:
            if l and l[0] in thai_letters:
                labels[typ][l] += 1
        for typ in ('fn', 'ln', 'title'):
            if typ in j and j[typ]:
                for qid in j[typ]:
                    qid = int(qid[6:])
                    labs = qid_lab_get(qid, 'th', include_alt=True).keys()
                    for l in labs:
                        if l and l[0] in thai_letters:
                            typ2 = typ + ('_th' if th else '')
                            labels[typ2][l] += 1



  0%|          | 0/25042218 [00:00<?, ?it/s]

In [67]:
th = False
if 'native_language' in j and 'WIKI_Q9217' in j['native_language']:
    th = True
elif 'name_native' in j and j['name_native'] and j['name_native'][0][0] in thai_letters:
    th = True
    
elif 'country' in j and 'WIKI_Q869' in j['country']:
    th = True
elif 'nationality' in j and 'WIKI_Q869' in j['nationality']:
    th = True
labs = list(qid_lab_get(qid, 'th', include_alt=True).keys())
if 'name_native' in j and j['name_native'] and j['name_native'][0][0] in thai_letters:
    labs += j['name_native']
for l in labs:
    if l and l[0] in thai_letters:
        print(l)

th, labs

ตู่
พล.อ.ประยุทธ์
ลุงตู่
พล.อ.ประยุทธ์ จันทร์โอชา
บิ๊กตู่
พลเอกประยุทธ์
พลเอก ประยุทธ์ จันทร์โอชา
พลเอกประยุทธ์ จันทร์โอชา
ประยุทธ์ จันทร์โอชา
ประยุทธ์ จันทร์โอชา


(True,
 ['ตู่',
  'พล.อ.ประยุทธ์',
  'ลุงตู่',
  'พล.อ.ประยุทธ์ จันทร์โอชา',
  'บิ๊กตู่',
  'พลเอกประยุทธ์',
  'พลเอก ประยุทธ์ จันทร์โอชา',
  'พลเอกประยุทธ์ จันทร์โอชา',
  'ประยุทธ์ จันทร์โอชา',
  'ประยุทธ์ จันทร์โอชา'])

In [62]:
import pickle
pickle.dump(labels, open('/projekti/thainame/data/wiki_names.pickle', 'wb'))

In [2]:
labels = pickle.load(open('/projekti/thainame/data/wiki_names.pickle', 'rb'))
for l in labs:
    print(l in labels['per_th'])

NameError: name 'pickle' is not defined

In [1]:
labels.key()

NameError: name 'labels' is not defined